<a href="https://colab.research.google.com/github/granantuin/Model_vs_data_label/blob/master/tensorflow_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from sklearn.metrics import confusion_matrix ,accuracy_score,f1_score,recall_score
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)



In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
drive_4km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_4km_h24toh48_dir/"
drive_1km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/"
dr=[drive_4km,drive_1km]
y_data=pd.read_excel(dr[0]+"y_coron_dir.xlsx",index_col=0)
x_data=pd.read_csv(dr[0]+"x_coron.csv",index_col=0)
x_data=x_data.iloc[:,0:9]
y_data=y_data[y_data>0]# delete station errors
result = x_data.join(y_data, how='outer').dropna()
x_data=result.iloc[:,0:9]
y_data=result.iloc[:,9:10]

In [0]:
def evaluate():
  index=["E","N","NE","NW","S","SE","SW","W"]
  print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=index, columns=index))
  print("****************")
  print("Accuracy=","{:.2%}".format(accuracy_score(y_test, y_pred)),"// Model Accuracy=45%")
  results= precision_recall_fscore_support(y_test, y_pred, average=None, )
  df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
  df=df.set_index("W_DIR")
  print("Average precision =","{:.2%}".format(df["Precision"].mean()),"// Model precision=28%")
  print("Average recall =","{:.2%}".format(df["Recall"].mean()),"// Model recall=28%")
  results= precision_recall_fscore_support(y_test, y_pred, average='weighted', )
  print("Precision weighted=","{:.2%}".format(results[0]),"//Model weighted=56%")
  print("Recall weighted =","{:.2%}".format(results[1]),"//Model weighted=53%")
  print("****************")
  print(df)

**E=[1,0,0,0,0,0,0,0]
N=[0,1,0,0,0,0,0,0]
NE=[0,0,1,0,0,0,0,0]
NW=[0,0,0,1,0,0,0,0]
S=[0,0,0,0,1,0,0,0,0]
SE=[0,0,0,0,0,1,0,0]
SW=[0,0,0,0,0,0,1,0]
W=[0,0,0,0,0,0,0,1]**

In [0]:
y_data_tf=[]
for dir in y_data["value"]:
  if dir>22.5 and dir<67.5:
    y_data_tf.append([0,0,1,0,0,0,0,0])
  if dir>67.5 and dir<112.5:
    y_data_tf.append([1,0,0,0,0,0,0,0])
  if dir>112.5 and dir<157.5:
    y_data_tf.append([0,0,0,0,0,1,0,0])
  if dir>157.5 and dir<202.5:
    y_data_tf.append([0,0,0,0,1,0,0,0])
  if dir>202.5 and dir<247.5:
    y_data_tf.append([0,0,0,0,0,0,1,0]) 
  if dir>247.5 and dir<292.5:
    y_data_tf.append([0,0,0,0,0,0,0,1])
  if dir>292.5 and dir<337.5:
    y_data_tf.append([0,0,0,1,0,0,0,0])
  if dir>337.5 or dir<22.5:
    y_data_tf.append([0,1,0,0,0,0,0,0])  
y_data=np.asarray(y_data_tf)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data, test_size=0.2, random_state=5)

In [0]:
l0 = tf.keras.layers.Dense(units=5, input_shape=[9]) 
l2 = tf.keras.layers.Dense(units=8,)
model = tf.keras.Sequential([l0,l2])
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(0.1))
model.summary()


In [0]:
history = model.fit(x_train, y_train, epochs=200, verbose=False)
print("Finished training the model")

In [0]:
hist=pd.DataFrame(history.history)

In [0]:
hist.plot()

In [0]:
y_pred=model.predict(x_test)


E=[1,0,0,0,0,0,0,0] N=[0,1,0,0,0,0,0,0] NE=[0,0,1,0,0,0,0,0] NW=[0,0,0,1,0,0,0,0] S=[0,0,0,0,1,0,0,0,0] SE=[0,0,0,0,0,1,0,0] SW=[0,0,0,0,0,0,1,0] W=[0,0,0,0,0,0,0,1]

In [0]:
y_pred_str=[]
for c in y_pred:
  if np.argmax(c)==0:
    y_pred_str.append("E")
  if np.argmax(c)==1:
    y_pred_str.append("N")
  if np.argmax(c)==2:
    y_pred_str.append("NE")
  if np.argmax(c)==3:
    y_pred_str.append("NW")
  if np.argmax(c)==4:
    y_pred_str.append("S")
  if np.argmax(c)==5:
    y_pred_str.append("SE")
  if np.argmax(c)==6:
    y_pred_str.append("SW")
  if np.argmax(c)==7:
    y_pred_str.append("W")  
y_pred=np.asarray(y_pred_str)

y_test_str=[]
for c in y_test:
  if np.argmax(c)==0:
    y_test_str.append("E")
  if np.argmax(c)==1:
    y_test_str.append("N")
  if np.argmax(c)==2:
    y_test_str.append("NE")
  if np.argmax(c)==3:
    y_test_str.append("NW")
  if np.argmax(c)==4:
    y_test_str.append("S")
  if np.argmax(c)==5:
    y_test_str.append("SE")
  if np.argmax(c)==6:
    y_test_str.append("SW")
  if np.argmax(c)==7:
    y_test_str.append("W")  
y_test=np.asarray(y_test_str)

In [0]:
evaluate()

**Overfit?**

In [0]:
y_test=y_train
y_pred=model.predict(x_train)
y_pred_str=[]
for c in y_pred:
  if np.argmax(c)==0:
    y_pred_str.append("E")
  if np.argmax(c)==1:
    y_pred_str.append("N")
  if np.argmax(c)==2:
    y_pred_str.append("NE")
  if np.argmax(c)==3:
    y_pred_str.append("NW")
  if np.argmax(c)==4:
    y_pred_str.append("S")
  if np.argmax(c)==5:
    y_pred_str.append("SE")
  if np.argmax(c)==6:
    y_pred_str.append("SW")
  if np.argmax(c)==7:
    y_pred_str.append("W")  
y_pred=np.asarray(y_pred_str)

y_test_str=[]
for c in y_test:
  if np.argmax(c)==0:
    y_test_str.append("E")
  if np.argmax(c)==1:
    y_test_str.append("N")
  if np.argmax(c)==2:
    y_test_str.append("NE")
  if np.argmax(c)==3:
    y_test_str.append("NW")
  if np.argmax(c)==4:
    y_test_str.append("S")
  if np.argmax(c)==5:
    y_test_str.append("SE")
  if np.argmax(c)==6:
    y_test_str.append("SW")
  if np.argmax(c)==7:
    y_test_str.append("W")  
y_test=np.asarray(y_test_str)


#evaluate()

In [0]:
evaluate()